In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
dic = {'위원회':[],
       '사건번호':[],
       '판정일':[],
       '결과':[],
       '내용':[],
       }



def crwaling(search_word) : 

    # For parameter setting of Lastpage Number
    url = 'http://www.nlrc.go.kr/nlrc/md/search_case.go'

    headers = {
        'Referer' : 'http://www.nlrc.go.kr/nlrc/md/search_case.go',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36 Edg/100.0.1185.36'}

    data = {     
            'inputChk': 'it',
            'pQuery_tmp': 'it',
            'even_gubn':'',
            'currentPage':'1',
            'searchGubn': 'BH',
            'pQuery': search_word,
            'key_end_fdate':'',
            'key_end_tdate':'',
            'key_comm_code': ''
            }

    resp = requests.post(url, headers=headers, data=data) # Post 방식
    soup = BeautifulSoup(resp.content, 'lxml')

    number = soup.select('span.content') # 페이지당 문서 갯수 (마지막 페이지의 문서 갯수 조절을 위해)
    search = soup.select('div.text_list p') 
    search_num = search[0].text.split(' ')[3][1:-1]
    a = soup.select('span.content')

    if int(search_num)%10 == 0 :
        last_page = int(search_num)//10
    else :
        last_page = int(search_num)//10 +1



    for page in tqdm(range(1, int(last_page)+1)):


        headers = {
            'Referer' : 'http://www.nlrc.go.kr/nlrc/md/search_case.go',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36 Edg/100.0.1185.36',
            'Cookie': 'JSESSIONID=9zd4WB36tREOxyh1r7FKNLDvYl1L6lTsO6dBwyhNkWFolAEr7QNjxwyZyx2zMkyd.amV1c19kb21haW4vbmxyYzE='}

        data = {
            'inputChk': 'it',
            'pQuery_tmp': 'it',
            'even_gubn':'',
            'currentPage':str(page),
            'searchGubn': 'BH',
            'pQuery': search_word,
            'key_end_fdate':'',
            'key_end_tdate':'',
            'key_comm_code': ''
            }


        resp = requests.post(url, headers=headers, data=data)
        soup = BeautifulSoup(resp.content, 'lxml')


        # too many requests make error
        try:
            resp = requests.post(url, headers=headers, data=data)
            soup = BeautifulSoup(resp.content, 'lxml')

        except requests.exceptions.ConnectionError:
            resp.status_code = "Connection refused"





    for i in range(len(number)):
        event = soup.select('div.text_list a')[i]['onclick'].replace("'",'').split(',')[2]  
        where = soup.select('div.text_list a')[i]['onclick'].replace("'",'').split(',')[1]  

        dic['위원회'].append(soup.select('div.text_list a')[i]['onclick'].replace("'",'').split(',')[1])
        dic['사건번호'].append(soup.select('div.text_list a')[i]['onclick'].replace("'",'').split(',')[2])
        dic['판정일'].append(soup.select('span.search_date')[i].text.strip())
        dic['결과'].append(soup.select('span.txt_blue')[1::2][i].text.split(' ')[1])

        new_url = 'http://www.nlrc.go.kr/nlrc/work/ad_mng_sum_q2_pop.go?even_gubn=JR01&comm_code='+str(where)+'&even_numb='+str(event)+'&resu_yeno=N&midd_rscd=20&r_merge_yesno=N&r_rep_merge_yesno=N&r_link_even_numb=&ptitle=%EA%B2%80%EC%83%89%EA%B2%B0%EA%B3%BC%EC%83%81%EC%84%B8%EB%B3%B4%EA%B8%B0'

        headers = {
            'referer' : 'http://www.nlrc.go.kr/nlrc/md/search_case.go',
            'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'
        }

        params = {
            'even_gubn': '',
            'comm_code': str(where),
            'even_numb': str(event),
            'resu_yeno': 'N',
            'midd_rscd': '20',
            'r_merge_yesno': 'N',
            'r_rep_merge_yesno': 'N',
            'r_link_even_numb': '',
            'ptitle': '검색결과상세보기'}

        new_resp = requests.get(new_url, headers=headers, params=params)
        new_soup = BeautifulSoup(new_resp.content, 'lxml')
        new_tags = new_soup.select('div#contents table.bbs_default_view02')

        dic['내용'].append(new_tags[0].text.split('판정요지')[2].replace('\n','').replace('\r',''))

    data = pd.DataFrame(dic)
    
    return data
    # for excel -> data.to_excel('크롤링.xlsx', index=False)
